In [1]:
from cltk.lemmatize.latin.backoff import BackoffLatinLemmatizer

In [2]:
lemmatizer = BackoffLatinLemmatizer()


In [3]:
from  data.word_tokenized_text import word_tokenized_text as text

In [4]:
lemmatized = lemmatizer.lemmatize(text)

In [5]:
lemmatized[:30]

[('Conditi', 'Conditi'),
 ('paradoxi', 'paradoxus'),
 ('compositio', 'compositio'),
 ('mellis', 'mel'),
 ('pondo', 'pondo'),
 ('XV', 'XV'),
 ('in', 'in'),
 ('aeneum', 'aeneus'),
 ('vas', 'vas'),
 ('mittuntur', 'mitto'),
 ('praemissis', 'praemitto'),
 ('vini', 'vinum'),
 ('sextariis', 'sextarius'),
 ('duobus', 'duo'),
 ('ut', 'ut'),
 ('in', 'in'),
 ('coctura', 'coquo'),
 ('mellis', 'mel'),
 ('vinum', 'vinum'),
 ('decoquas', 'decoquo'),
 ('quod', 'qui'),
 ('igni', 'ignis'),
 ('lento', 'lentus'),
 ('et', 'et'),
 ('aridis', 'aridus'),
 ('lignis', 'lignum'),
 ('calefactum', 'calefacio'),
 ('commotum', 'commoveo'),
 ('ferula', 'ferula'),
 ('dum', 'dum')]